In [ ]:
##modules
#%matplotlib widget
#%matplotlib inline
#
#%matplotlib qt
import mne
import numpy as np
import matplotlib
# Establecer un backend interactivo, como 'Qt5Agg', 'GTK3Agg', etc.
# Esto depende de los backends disponibles en tu sistema.

import matplotlib.pyplot as plt

matplotlib.use('TkAgg')  # Asegúrate de que este backend está instalado.

import pandas as pd 
import os
import sys

from mne.preprocessing import ICA, corrmap, create_ecg_epochs, create_eog_epochs

from os.path import join as pathjoin
from time import time

from pathlib import Path

from autoreject import AutoReject

In [ ]:
# Variables path
layer_script = "block"

#process=""
disco="g"

# subj = "sub-A2004"

#subj = sys.argv[1] ## name of participant list

# Carpeta general
datadir = Path(f"{disco}:\\PROYECTO_SELF")

# #carpetas generales de datos
brain_vision_dir = datadir /"Data_self"/"Export_acw_Brain_Vision"
fif_data = datadir /"Data_self"/ "fif_data" 


#carpeta analysis
output_analysis= datadir / "output_analysis"

acw_analysis_path = output_analysis / "acw"
acw_analysis_path.mkdir(parents=True, exist_ok=True)

# # Carpeta de preprocesado
# output_preproc = datadir / "output_preproc"

# preproc_path = output_preproc / f"preproc_{layer_script}"
# preproc_path.mkdir(parents=True, exist_ok=True)
 
# # Carpeta de epocas "sucias"
# epochs_path = preproc_path / f"epochs_{layer_script}"
# epochs_path.mkdir(parents=True, exist_ok=True) 

# # Carpeta de ICA
# ICA_path = preproc_path / f"ICA_{layer_script}"
# ICA_path.mkdir(parents=True, exist_ok=True)

# # Épocas limpias
# epochs_clean_path = preproc_path / f"epochs_clean_{layer_script}"
# epochs_clean_path.mkdir(parents=True, exist_ok=True)

# #epocas evoked
# evoked_path = Path(preproc_path) / f"evoked_{layer_script}"
# evoked_path.mkdir(parents=True, exist_ok=True)
 

# # Definir la carpeta de output_source antes de usarla
# output_source = Path(r"g:\MOUS_204\output_source")

# source_path = output_source / f"source_{layer_script}"
# source_path.mkdir(parents=True, exist_ok=True)

# #raw_hsp es el raw con fiducials cargados
# raw_hsp_path = source_path / f"raw_hsp"
# raw_hsp_path.mkdir(parents=True, exist_ok=True)

# # Carpeta de forward solution
# fwd_path = source_path / f"fwd"
# fwd_path.mkdir(parents=True, exist_ok=True)

# # Carpeta de inverse solution
# inverse_path = source_path / f"inverse"
# inverse_path.mkdir(parents=True, exist_ok=True)


# mne.utils.set_config('SUBJECTS_DIR', r'\\wsl$\Ubuntu-20.04\usr\local\freesurfer\subjects', set_env=True)


In [ ]:

##EVOKEDS


# # epochs_zinnen_path = epochs_clean_path / f"{subj}_epochs_zinnen_{type_script}"
# # epochs_woorden_path = epochs_clean_path / f"{subj}_epochs_woorden_{type_script}"

# path_zinnen= evoked_path / f"{subj}_evoked_zinnen_{layer_script}-ave.fif"

# evokeds_zinnen = mne.read_evokeds(path_zinnen) 

# evoked_zinnen=evokeds_zinnen[0]



# path_woorden= evoked_path / f"{subj}_evoked_woorden_{layer_script}-ave.fif"

# evokeds_woorden = mne.read_evokeds(path_woorden) 

# evoked_woorden=evokeds_woorden[0]


In [ ]:
# aplicar la acf EN EVOKED

from statsmodels.tsa.stattools import acf
import numpy as np

def acf_blocks(evoked, isplot=False): 
    
    #import evoked object and get data ONLY ON MEG DATA
    data_evoked=evoked.copy().pick("meg").get_data()

    #get duration
    duration= evoked.tmax - evoked.tmin
    #get sample frequency
    sfreq= evoked.info['sfreq']
    #get lags
    lags=duration*sfreq

    del evoked
  
   #list to store the values of acf, acw_50,acw_0 for each sensor
    acw_50_elect_all_s = []
    acw_0_elect_all_s = []
    acf_elect_all = []

    #calculus of value  for EACH SENSOR
    for i in range(0,len(data_evoked)):
          #lags se puede dejar por defecto porque te cogelos valores
    #  hasta los que tiene sentido calcularlo, aunque yo voy a calcularlo para todos los lags
        #calculo de acf de sensor
        #alfa lo dejo endefault que es la confianza del 95% 

        acf_elect, qstat_vals, pvals = acf(
            data_evoked[i],
            adjusted=True,
            fft=True,
            qstat=True,
            nlags=lags
        )

        acf_elect_all.append(acf_elect)

        #calculo de acw_50 y acw_0 de sensor for both in lags and in seconds
        acw_50_elect_lags = np.argmax(acf_elect <= 0.5)
        acw_50_elect_s = acw_50_elect_lags / sfreq
        acw_50_elect_all_s.append(acw_50_elect_s)


        acw_0_elect_lags = np.argmax(acf_elect <= 0)
        acw_0_elect_s = acw_0_elect_lags / sfreq
        acw_0_elect_all_s.append(acw_0_elect_s)

    # #transformation of list to array
    # acf_elect_all=np.array(acf_elect_all)
    # acw_50_elect_all_s=np.array(acw_50_elect_all_s)
    # acw_0_elect_all_s=np.array(acw_0_elect_all_s)



    #identificar acw_0


    #identificar acw_50


    #valores de estadística
        ## pero ver que valores son puto significativos de la ACF o de la ACW??



    isplot=False    

    # if isplot==True:
    #     # 1) Creamos un vector de lags en segundos,
    #     #    asumiendo que acf_val tiene tantos puntos como lags + 1.
    #     time_lags = np.arange(len(acf_val)) / sfreq

    #     # 2) Convertimos los índices (entero) de ACW_50 y ACW_0
    #     ACW_50_i = int(acw_50_lags)
    #     ACW_0_i = int(acw_0_lags)

    #     # 3) Figura
    #     plt.figure(figsize=(8, 4))

    #     # 4) Dibujamos la ACF en negro
    #     plt.plot(time_lags, acf_val, 'k', label='ACF')

    #     # 5) Acotamos el eje X a la duración total de la señal (o a lo que consideres)
    #     plt.xlim([0, time_lags[-1]])

    #     # 6) Rellenamos el área hasta ACW_50
    #     plt.fill_between(
    #         time_lags[:ACW_50_i + 1],
    #         acf_val[:ACW_50_i + 1],
    #         color='r', alpha=0.3, label='ACW-50 area'
    #     )

    #     # 7) Rellenamos el área hasta ACW_0
    #     plt.fill_between(
    #         time_lags[:ACW_0_i + 1],
    #         acf_val[:ACW_0_i + 1],
    #         color='m', alpha=0.3, label='ACW-0 area'
    #     )

    #     # 8) Título con valores de ACW
    #     plt.title(f'ACW-0 = {acw_0_s:.1f} s    ACW-50 = {acw_50_s:.1f} s')
    #     plt.xlabel('Lags (s)')
    #     plt.ylabel('Autocorrelation')
    #     plt.legend(loc='best')
    #     plt.show()

    return acf_elect_all,acw_50_elect_all_s, acw_0_elect_all_s



In [145]:
evoked_zinnen_crop=evoked_zinnen.copy().crop(tmin=0.0, tmax=38)

In [159]:
acf_elect_all_zinnen,acw_50_elect_all_s_zinnen, acw_0_elect_all_s_zinnen= acf_blocks(evoked_zinnen_crop, isplot=False)

In [150]:
print(f"zinnen acw_50_elect_all_s.mean()= {acw_50_elect_all_s.mean()}")

print(f"woorden acw_0_elect_all_s.mean()= {acw_0_elect_all_s.mean()}")



zinnen acw_50_elect_all_s.mean()= 0.019975579975579975
woorden acw_0_elect_all_s.mean()= 0.3116666666666667


In [ ]:


evoked_woorden_crop=evoked_woorden.copy().crop(tmin=0.0, tmax=38)


Reading g:\MOUS_204\output_preproc\preproc_block\evoked_block\sub-A2004_evoked_woorden_block-ave.fif ...
    Reading extended channel information
    Found the data of interest:
        t =       0.00 ...   54000.00 ms (fix_woorden)
        0 CTF compensation matrices available
        nave = 24 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
No baseline correction applied


In [157]:
acf_elect_all_woorden.shape

(273, 22801)

In [160]:
acf_elect_all_woorden,acw_50_elect_all_s_woorden, acw_0_elect_all_s_woorden= acf_blocks(evoked_woorden_crop, isplot=False)

In [153]:
print(f"woorden acw_50_elect_all_s.mean()= {acw_50_elect_all_s.mean()}")

print(f"woorden  acw_0_elect_all_s.mean()= {acw_0_elect_all_s.mean()}")


woorden acw_50_elect_all_s.mean()= 0.019853479853479853
woorden  acw_0_elect_all_s.mean()= 0.2765873015873016


In [177]:
import pandas as pd

# Crear un DataFrame con las columnas necesarias
df_acf = pd.DataFrame({
    'Subject': subj,  # Asumiendo que subj es una lista de nombres de sujetos
    "Sensor": range(1,len(data_evoked)+1),
    'ACF_Zinnen': acf_elect_all_zinnen,
    'ACW_50_Zinnen': acw_50_elect_all_s_zinnen,
    'ACW_0_Zinnen': acw_0_elect_all_s_zinnen,
    'ACF_Woorden': acf_elect_all_woorden,
    'ACW_50_Woorden': acw_50_elect_all_s_woorden,
    'ACW_0_Woorden': acw_0_elect_all_s_woorden
})

In [178]:
df_acf

,Subject,Sensor,ACF_Zinnen,ACW_50_Zinnen,ACW_0_Zinnen,ACF_Woorden,ACW_50_Woorden,ACW_0_Woorden
0,sub-A2004,1,"[1.0, 0.9905048442203288, 0.9626894588404757, ...",0.016667,0.395000,"[1.0, 0.9895735075857295, 0.9591936537759487, ...",0.016667,0.343333
1,sub-A2004,2,"[1.0, 0.9911062430568643, 0.9650612053071513, ...",0.018333,0.361667,"[1.0, 0.9903379625234613, 0.9621003464625999, ...",0.016667,0.351667
2,sub-A2004,3,"[1.0, 0.9913972044597323, 0.9661996970288781, ...",0.018333,0.398333,"[1.0, 0.990714713974279, 0.9635488534795714, 0...",0.018333,0.351667
3,sub-A2004,4,"[1.0, 0.990974730770809, 0.9645398933929792, 0...",0.016667,0.471667,"[1.0, 0.990637266137112, 0.9632189858315947, 0...",0.016667,0.328333
4,sub-A2004,5,"[1.0, 0.9916254312710292, 0.9670498784935911, ...",0.018333,0.468333,"[1.0, 0.9912676287131756, 0.9656590423899697, ...",0.018333,0.266667
...,...,...,...,...,...,...,...,...
268,sub-A2004,269,"[1.0, 0.9889935092032794, 0.9568210882016889, ...",0.015000,0.351667,"[1.0, 0.9872562112907339, 0.9501653838989425, ...",0.013333,0.263333
269,sub-A2004,270,"[1.0, 0.9917192296075656, 0.967348161474492, 0...",0.016667,0.041667,"[1.0, 0.9917844540267222, 0.9675631467637682, ...",0.016667,0.033333
270,sub-A2004,271,"[1.0, 0.9914043659396727, 0.9661165824128212, ...",0.016667,0.035000,"[1.0, 0.9911004369607459, 0.9649506438804925, ...",0.016667,0.033333
271,sub-A2004,272,"[1.0, 0.9899884625152103, 0.9606623073355224, ...",0.016667,0.346667,"[1.0, 0.9895155098640743, 0.9588877184648875, ...",0.015000,0.041667


In [ ]:
##code to save the dataframe 

df_acf.to_csv(analysis_path / f"acf_{layer_script}.csv", index=False)

In [ ]:
# code to read the dataframe

df_acf_loaded = pd.read_csv(analysis_path / f"acf_{layer_script}.csv")


In [186]:
df_acf_loaded

,Subject,Sensor,ACF_Zinnen,ACW_50_Zinnen,ACW_0_Zinnen,ACF_Woorden,ACW_50_Woorden,ACW_0_Woorden
0,sub-A2004,1,[ 1.00000000e+00 9.90504844e-01 9.62689459e-...,0.016667,0.395000,[ 1. 0.98957351 0.95919365 ... -1.02...,0.016667,0.343333
1,sub-A2004,2,[ 1. 0.99110624 0.96506121 ... -0.07...,0.018333,0.361667,[ 1. 0.99033796 0.96210035 ... 0.00...,0.016667,0.351667
2,sub-A2004,3,[ 1. 0.9913972 0.9661997 ... -0.21...,0.018333,0.398333,[ 1. 0.99071471 0.96354885 ... -0.08...,0.018333,0.351667
3,sub-A2004,4,[ 1. 0.99097473 0.96453989 ... -0.14...,0.016667,0.471667,[1. 0.99063727 0.96321899 ... 0.081492...,0.016667,0.328333
4,sub-A2004,5,[ 1. 0.99162543 0.96704988 ... -0.01...,0.018333,0.468333,[1. 0.99126763 0.96565904 ... 0.058078...,0.018333,0.266667
...,...,...,...,...,...,...,...,...
268,sub-A2004,269,[ 1. 0.98899351 0.95682109 ... -0.01...,0.015000,0.351667,[ 1. 0.98725621 0.95016538 ... -1.19...,0.013333,0.263333
269,sub-A2004,270,[ 1. 0.99171923 0.96734816 ... -0.01...,0.016667,0.041667,[1. 0.99178445 0.96756315 ... 0.375049...,0.016667,0.033333
270,sub-A2004,271,[ 1. 0.99140437 0.96611658 ... -0.32...,0.016667,0.035000,[1. 0.99110044 0.96495064 ... 0.241846...,0.016667,0.033333
271,sub-A2004,272,[1. 0.98998846 0.96066231 ... 0.008007...,0.016667,0.346667,[1. 0.98951551 0.95888772 ... 0.563044...,0.015000,0.041667


In [ ]:
#in case you want to add a new subjet to the dataframe its better to concatenate it 

In [190]:
# if you want a loop


# Lista donde almacenaremos los datos de cada sujeto antes de convertirlo en DataFrame
data_list = []
subjects = [ 'sub-A2004', 'sub-A2005']


# Iteramos sobre cada sujeto en la lista 'subjects'
for subj in subjects:

        # epochs_zinnen_path = epochs_clean_path / f"{subj}_epochs_zinnen_{type_script}"
    # epochs_woorden_path = epochs_clean_path / f"{subj}_epochs_woorden_{type_script}"

    path_zinnen= evoked_path / f"{subj}_evoked_zinnen_{layer_script}-ave.fif"

    evokeds_zinnen = mne.read_evokeds(path_zinnen) 

    evoked_zinnen=evokeds_zinnen[0]
    #delete this line
    evoked_zinnen_crop=evoked_zinnen.crop(tmin=0.0, tmax=38)




    path_woorden= evoked_path / f"{subj}_evoked_woorden_{layer_script}-ave.fif"

    evokeds_woorden = mne.read_evokeds(path_woorden) 

    evoked_woorden=evokeds_woorden[0]
    evoked_woorden_crop=evoked_woorden.crop(tmin=0.0, tmax=38)
    # Calculamos ACF para Zinnen y Woorden
    acf_elect_all_zinnen, acw_50_elect_all_s_zinnen, acw_0_elect_all_s_zinnen = acf_blocks(evoked_zinnen_crop, isplot=False)
    acf_elect_all_woorden, acw_50_elect_all_s_woorden, acw_0_elect_all_s_woorden = acf_blocks(evoked_woorden_crop, isplot=False)

    # Iteramos sobre cada sensor
    for sensor_id in range(1, len(acf_elect_all_zinnen) + 1):
        data_list.append({
            'Subject': subj,
            'Sensor': sensor_id,
            'ACF_Zinnen': acf_elect_all_zinnen[sensor_id - 1],
            'ACW_50_Zinnen': acw_50_elect_all_s_zinnen[sensor_id - 1],
            'ACW_0_Zinnen': acw_0_elect_all_s_zinnen[sensor_id - 1],
            'ACF_Woorden': acf_elect_all_woorden[sensor_id - 1],
            'ACW_50_Woorden': acw_50_elect_all_s_woorden[sensor_id - 1],
            'ACW_0_Woorden': acw_0_elect_all_s_woorden[sensor_id - 1]
        })
    
    del evokeds_zinnen, evoked_zinnen, evokeds_woorden, evoked_woorden

# Convertimos la lista de diccionarios en un DataFrame
df_acf = pd.DataFrame(data_list)

Reading g:\MOUS_204\output_preproc\preproc_block\evoked_block\sub-A2004_evoked_zinnen_block-ave.fif ...
    Reading extended channel information
    Found the data of interest:
        t =       0.00 ...   42000.00 ms (fix_zinnen)
        0 CTF compensation matrices available
        nave = 24 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
No baseline correction applied
Reading g:\MOUS_204\output_preproc\preproc_block\evoked_block\sub-A2004_evoked_woorden_block-ave.fif ...
    Reading extended channel information
    Found the data of interest:
        t =       0.00 ...   54000.00 ms (fix_woorden)
        0 CTF compensation matrices available
        nave = 24 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
No baseline correction applied
Reading g:\MOUS_204\output_preproc\preproc_block\evoked_block\sub-A2005_evoked_zinnen_block-ave.fif ...
    Reading extended channel informa

In [191]:
df_acf

,Subject,Sensor,ACF_Zinnen,ACW_50_Zinnen,ACW_0_Zinnen,ACF_Woorden,ACW_50_Woorden,ACW_0_Woorden
0,sub-A2004,1,"[1.0, 0.9905048442203288, 0.9626894588404757, ...",0.016667,0.395000,"[1.0, 0.9895735075857295, 0.9591936537759487, ...",0.016667,0.343333
1,sub-A2004,2,"[1.0, 0.9911062430568643, 0.9650612053071513, ...",0.018333,0.361667,"[1.0, 0.9903379625234613, 0.9621003464625999, ...",0.016667,0.351667
2,sub-A2004,3,"[1.0, 0.9913972044597323, 0.9661996970288781, ...",0.018333,0.398333,"[1.0, 0.990714713974279, 0.9635488534795714, 0...",0.018333,0.351667
3,sub-A2004,4,"[1.0, 0.990974730770809, 0.9645398933929792, 0...",0.016667,0.471667,"[1.0, 0.990637266137112, 0.9632189858315947, 0...",0.016667,0.328333
4,sub-A2004,5,"[1.0, 0.9916254312710292, 0.9670498784935911, ...",0.018333,0.468333,"[1.0, 0.9912676287131756, 0.9656590423899697, ...",0.018333,0.266667
...,...,...,...,...,...,...,...,...
541,sub-A2005,269,"[1.0, 0.9858978615635348, 0.9448252353092444, ...",0.013333,0.280000,"[1.0, 0.9873623389402046, 0.9505504044893055, ...",0.013333,0.356667
542,sub-A2005,270,"[1.0, 0.9911123603599127, 0.964886679055864, 0...",0.015000,0.030000,"[1.0, 0.9916731635909755, 0.9671543759534882, ...",0.016667,0.030000
543,sub-A2005,271,"[1.0, 0.9906314293169298, 0.9631338340489339, ...",0.015000,0.028333,"[1.0, 0.9913487706744277, 0.965965420723465, 0...",0.016667,0.030000
544,sub-A2005,272,"[1.0, 0.9859497479112347, 0.9452889334180269, ...",0.013333,0.035000,"[1.0, 0.9876087013820392, 0.9516930474300026, ...",0.013333,0.038333
